In [1]:
############################################################################################################
# Overview: This script encodes the extracted patient-level data mart.
############################################################################################################

In [2]:
############################################################################################################
# Import packages
############################################################################################################
import datetime
import numpy as np
import os
import pandas as pd
import warnings
from ast import literal_eval
warnings.filterwarnings('ignore', category=pd.errors.SettingWithCopyWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

In [6]:
########################################################################################################################
# USER_SPECIFIC SETTING
# DATA_IN_DIR_PATH: Path of the input directory of the patient-level dataset 
# (created in C01_Data_Transfer.ipynb)
# DICT_OUT_DIR_PATH: Path of the output directory to store the data dictionary   
########################################################################################################################
DATA_IN_DIR_PATH: str = '../00_Data/00_Raw_Data/'
DICT_OUT_DIR_PATH: str = '../00_Data/99_Dictionary/'

In [ ]:
########################################################################################################################
# Extract the variables in the data
########################################################################################################################
df_pat: pd.DataFrame = pd.read_parquet(os.path.join(DATA_IN_DIR_PATH, 'Patient_full.parquet'))
pat_vars: list[str] = df_pat.drop(columns=['PatientDurableKey']).columns.to_list()

In [3]:
########################################################################################################################
# Create a data dictionary in pandas.DataFrame format
########################################################################################################################
df_pat_dict: pd.DataFrame = pd.DataFrame({'Variable_Name': pat_vars})
df_pat_dict['Encoded_Values'] = np.nan
df_pat_dict['Variable_Type'] = np.nan
df_pat_dict['Remark'] = np.nan

In [ ]:
########################################################################################################################
# 1. Encode date variables
########################################################################################################################
date_cols: list[str] = [pat_vars[i] for i in range(27, 104, 2)]
assert all(col.endswith('DateKey') for col in date_cols)
df_pat_dict.loc[df_pat_dict['Variable_Name'].isin(date_cols), 'Variable_Type'] = 'Date as float'

In [ ]:
########################################################################################################################
# 2. Encode binary variables
########################################################################################################################
bin_cols: list[str] = [pat_vars[i] for i in range(104, 128)]
for b in bin_cols:
    assert set(df_pat[b].unique().tolist()) == {'Y', 'N'}
df_pat_dict.loc[df_pat_dict['Variable_Name'].isin(bin_cols), 'Variable_Type'] = 'Binary'
df_pat_dict.loc[df_pat_dict['Variable_Name'].isin(bin_cols), 'Encoded_Values'] = str({'N': 0, 'Y': 1})

In [ ]:
########################################################################################################################
# 3. Manual date-time encoding
########################################################################################################################
datetime_cols: list[str] = ['BirthDate', 'DeathDate', ]
df_pat_dict.loc[df_pat_dict['Variable_Name'].isin(datetime_cols), 'Variable_Type'] = 'datetime.date'

In [ ]:
########################################################################################################################
# 4. Race encoding
########################################################################################################################
race_cols: list[str] = ['FirstRace', 'SecondRace', 'ThirdRace', 'FourthRace', 'FifthRace']
encode: dict = {'': None}
unique_race: list[str] = df_pat['FirstRace'].unique().tolist()
unique_race = sorted([k for k in unique_race if k not in encode.keys()])
encode |= {i: idx for idx, i in enumerate(unique_race)}
df_pat_dict.loc[df_pat_dict['Variable_Name'].isin(race_cols), 'Variable_Type'] = 'Nominal'
df_pat_dict.loc[df_pat_dict['Variable_Name'].isin(race_cols), 'Encoded_Values'] = str(encode)

In [ ]:
########################################################################################################################
# 5. Ethnicity encoding
########################################################################################################################
encode: dict = {'Unspecified': None}
encode |= {'Not Hispanic or Latino': 0, 'Hispanic or Latino': 1}
df_pat_dict.loc[df_pat_dict['Variable_Name'] == 'Ethnicity', 'Variable_Type'] = 'Binary'
df_pat_dict.loc[df_pat_dict['Variable_Name'] == 'Ethnicity', 'Encoded_Values'] = str(encode)

In [ ]:
########################################################################################################################
# 6. Sex encoding
########################################################################################################################
encode: dict = {'*Unspecified': None, '*Masked': None, 'Unknown': None}
encode |= {'Female': 0, 'Male': 1, 'Other': 2}
df_pat_dict.loc[df_pat_dict['Variable_Name'] == 'Sex', 'Variable_Type'] = 'Nominal'
df_pat_dict.loc[df_pat_dict['Variable_Name'] == 'Sex', 'Encoded_Values'] = str(encode)

In [ ]:
########################################################################################################################
# 7. Sex-assigned-at-birth encoding
########################################################################################################################
encode: dict = {'*Unspecified': None, 'Unknown': None, 'Choose not to disclose': -1}
encode |= {'Female': 0, 'Male': 1, 'Uncertain': 2, 'Not recorded on birth certificate': 3}
df_pat_dict.loc[df_pat_dict['Variable_Name'] == 'SexAssignedAtBirth', 'Variable_Type'] = 'Nominal'
df_pat_dict.loc[df_pat_dict['Variable_Name'] == 'SexAssignedAtBirth', 'Encoded_Values'] = str(encode)

In [ ]:
########################################################################################################################
# 8. Gender identity encoding
########################################################################################################################
encode: dict = {'*Unspecified': None, '*Masked': None, 'Choose not to disclose': -1}
encode |= {'Female': 0, 'Male': 1, 'Transgender Male / Female-to-Male': 2, 'Transgender Female / Male-to-Female': 3}
df_pat_dict.loc[df_pat_dict['Variable_Name'] == 'GenderIdentity', 'Variable_Type'] = 'Nominal'
df_pat_dict.loc[df_pat_dict['Variable_Name'] == 'GenderIdentity', 'Encoded_Values'] = str(encode)

In [ ]:
########################################################################################################################
# 9. One-hot encoding of other nominal variables
########################################################################################################################
ohe_cols: list[str] = ['PreferredLanguage', 
                       'ValidatedStateOrProvince_X', 
                       'MaritalStatus', 
                       'BirthControl',
                       'AbusedSubstance',
                       'TravelHistory',
                       'CommDiseaseExp']
for c in ohe_cols:
    encode: dict = {'*Unspecified': None, '*Masked': None, 'Unknown': None}
    u_vals: list[str] = df_pat[c].unique().tolist()
    u_vals = sorted([k for k in u_vals if k not in encode.keys() and k is not None])
    encode |= {i: idx for idx, i in enumerate(u_vals)}
    df_pat_dict.loc[df_pat_dict['Variable_Name'] == c, 'Variable_Type'] = 'Nominal'
    df_pat_dict.loc[df_pat_dict['Variable_Name'] == c, 'Encoded_Values'] = str(encode)

In [ ]:
########################################################################################################################
# 10a. Continuous variables encoding
########################################################################################################################
cont_cols: list[str] = ['SviHouseholdCharacteristicsPctlRankByZip2020_X',
                        'SviHouseholdCompositionPctlRankingByZip2018_X',
                        'SviHousingTypeTransportationPctlRankByZip2020_X',
                        'SviHousingTypeTransportationPctlRankingByZip2018_X',
                        'SviMinorityStatusLanguagePctlRankingByZip2018_X',
                        'SviOverallPctlRankByZip2020_X',
                        'SviOverallPctlRankingByZip2018_X',
                        'SviRacialEthnicMinorityStatusPctlRankByZip2020_X',
                        'SviSocioeconomicPctlRankByZip2020_X',
                        'SviSocioeconomicPctlRankingByZip2018_X',
                        'ADIUSPercentileRank',
                        'FreqDrugMisuse',
                        'CigPacksPerDay',
                        'HousingPlaceLived',
                       ]
df_pat_dict.loc[df_pat_dict['Variable_Name'].isin(cont_cols), 'Variable_Type'] = 'Continuous'

In [ ]:
########################################################################################################################
# 10b. Continuous variables encoding (but with unreasonably negative values)
########################################################################################################################
cont_cols: list[str] = ['AlcDrinksPerDay',
                        'CigPackYears',
                       ]
df_pat_dict.loc[df_pat_dict['Variable_Name'].isin(cont_cols), 'Variable_Type'] = 'Continuous'
df_pat_dict.loc[df_pat_dict['Variable_Name'].isin(cont_cols), 'Remark'] = 'Unreasonably negative values detected'

In [ ]:
########################################################################################################################
# 11. Ordinal variables encoding
########################################################################################################################
ord_dict: dict[str, dict] = {}
ord_dict['FoodWorry'] = {'Not Asked': None, 'Patient declined': -1, 'Patient unable to answer': -2,
                         'Never true': 0, 'Sometimes true': 1, 'Often true': 2}
ord_dict['HistoryAlcUse'] = {'Not Asked': None, 'Never': 0, 'No': 1, 'Not Currently': 2, 'Yes': 3}
ord_dict['SocConnPhone'] = {'Not Asked': None, 'Patient declined': -1, 'Patient unable to answer': -2,
                            'Never': 0, 'Once a week': 1, 'Twice a week': 2, 'Three times a week': 3, 'More than three times a week': 4}
ord_dict['FoodScarcity'] = {'Not Asked': None, 'Patient declined': -1, 'Patient unable to answer': -2,
                            'Never true': 0, 'Sometimes true': 1, 'Often true': 2}
ord_dict['SocConnGetTog'] = {'Not Asked': None, 'Patient declined': -1, 'Patient unable to answer': -2,
                             'Never': 0, 'Once a week': 1, 'Twice a week': 2, 'Three times a week': 3, 'More than three times a week': 4}
ord_dict['SmokingStatus'] = {'Unknown': None, 'Smoker, Current Status Unknown': -3, 'Never': 0, 'Former': 1,
                             'Light Smoker': 2, 'Some Days': 3, 'Every Day': 4, 'Heavy Smoker': 5}
ord_dict['PhysActivityDPW'] = {'Not Asked': None, 'Patient declined': -1, 'Patient unable to answer': -2, 
                               '0 days': 0, '1 day': 1, '2 days': 2, '3 days': 3,
                               '4 days': 4, '5 days': 5, '6 days': 6, '7 days': 7}
ord_dict['Stress'] = {'Patient unable to answer': -2, 'Not at all': 0, 'Only a little': 1,
                      'To some extent': 2, 'Rather much': 3, 'Very much': 4}
ord_dict['SocConnChurch'] = {'Not Asked': None, 'Patient declined': -1, 'Patient unable to answer': -2,
                             'Never': 0, '1 to 4 times per year': 1, 'More than 4 times per year': 2}
ord_dict['PhysActivityMPS'] = {'Not Asked': None, 'Patient declined': -1, 'Patient unable to answer': -2,
                               '0 min': 0, '10 min': 1, '20 min': 2, '30 min': 3, '40 min': 4,
                               '50 min': 5, '60 min': 6, '70 min': 7, '80 min': 8, '90 min': 9,
                               '100 min': 10, '110 min': 11, '120 min': 12, '130 min': 13, '140 min': 14,
                               '150+ min': 15}
ord_dict['SmokelessStatus'] = {'Never': 0, 'Former': 1, 'Current': 2}
ord_dict['SexuallyActive'] = {'Never': 0, 'Not Currently': 1, 'Yes': 2}
ord_dict['AlcStdDrinks'] =  {'Not Asked': None, 'Patient declined': -1, 'Patient unable to answer': -2,
                             'Patient does not drink': 0, '1 or 2': 1, '3 or 4': 2, '5 or 6': 3, '7 to 9': 4, '10 or more': 5}
ord_dict['Financial'] = {'Patient unable to answer': -2, 'Not hard at all': 0, 'Not very hard': 1,
                         'Somewhat hard': 2, 'Hard': 3, 'Very hard': 4}
ord_dict['AlcoholFreq'] = {'Not Asked': None, 'Patient declined': -1, 'Patient unable to answer': -2,
                           'Never': 0, 'Monthly or less': 1, '2-4 times a month': 2, '2-3 times a week': 3, '4 or more times a week': 4}
ord_dict['AlcoholBinge'] = {'Not Asked': None, 'Patient declined': -1, 'Patient unable to answer': -2,
                            'Never': 0, 'Less than monthly': 1, 'Monthly': 2, 'Weekly': 3, 'Daily or almost daily': 4}
ord_dict['SubstUseStatus'] = {'Not Asked': None, 'Never': 0, 'No': 1, 'Not Currently': 2, 'Yes': 3}
ord_dict['SocConnMeetings'] = {'Not Asked': None, 'Patient declined': -1, 'Patient unable to answer': -2,
                               'Never': 0, '1 to 4 times per year': 1, 'More than 4 times per year': 2}

for var, encode in ord_dict.items():
    df_pat_dict.loc[df_pat_dict['Variable_Name'] == var, 'Variable_Type'] = 'Ordinal'
    df_pat_dict.loc[df_pat_dict['Variable_Name'] == var, 'Encoded_Values'] = str(encode) 


In [ ]:
########################################################################################################################
# 12. Other binary variables encoding
########################################################################################################################
bin_dict: dict[str, dict] = {}
bin_dict['SocConnMember'] = {'Not Asked': None, 'Patient declined': -1, 'Patient unable to answer': -2,
                             'No': 0, 'Yes': 1}
bin_dict['TransportMed'] = {'Not Asked': None, 'Patient declined': -1, 'Patient unable to answer': -2,
                             'No': 0, 'Yes': 1}
bin_dict['IPVPhysAbuse'] = {'Not Asked': None, 'Patient declined': -1, 'Patient unable to answer': -2,
                            'No': 0, 'Yes': 1}
bin_dict['TransportNonMed'] = {'Not Asked': None, 'Patient declined': -1, 'Patient unable to answer': -2,
                               'No': 0, 'Yes': 1}
bin_dict['IPVEmotional'] = {'Not Asked': None, 'Patient declined': -1, 'Patient unable to answer': -2,
                            'No': 0, 'Yes': 1}
bin_dict['IPVFear'] = {'Not Asked': None, 'Patient declined': -1, 'Patient unable to answer': -2,
                       'No': 0, 'Yes': 1}
bin_dict['IPVSexualAbuse'] = {'Not Asked': None, 'Patient declined': -1, 'Patient unable to answer': -2,
                              'No': 0, 'Yes': 1}
bin_dict['SexualPartner'] = {'Female': 0, 'Male': 1}
bin_dict['HousingHomeless'] = {'Patient declined': -1, 'Patient unable to answer': -2, 'No': 0, 'Yes': 1}
bin_dict['HousingMortgage'] = {'Patient declined': -1, 'Patient unable to answer': -2, 'No': 0, 'Yes': 1}

for var, encode in bin_dict.items():
    df_pat_dict.loc[df_pat_dict['Variable_Name'] == var, 'Variable_Type'] = 'Binary'
    df_pat_dict.loc[df_pat_dict['Variable_Name'] == var, 'Encoded_Values'] = str(encode) 

In [ ]:
########################################################################################################################
# 13. Variables to drop (unneeded or duplicated)
########################################################################################################################
dup_cols: list[str] = ['ValidatedStateOrProvinceAbbreviation_X']
df_pat_dict.loc[df_pat_dict['Variable_Name'].isin(dup_cols), 'Variable_Type'] = 'DROP'

In [ ]:
########################################################################################################################
# 14. Constants
########################################################################################################################
var = 'SocConnLiving'
encode = {'Not Asked': None, 'Patient declined': -1, 'Patient unable to answer': -2, 'Separated': 0}
df_pat_dict.loc[df_pat_dict['Variable_Name'] == var, 'Variable_Type'] = 'Constant'
df_pat_dict.loc[df_pat_dict['Variable_Name'] == var, 'Encoded_Values'] = str(encode)

In [ ]:
########################################################################################################################
# 15. Variables with no data
########################################################################################################################
empty_cols: list[str] = ['TobaccoUse']
df_pat_dict.loc[df_pat_dict['Variable_Name'].isin(empty_cols), 'Variable_Type'] = 'No Data'

In [ ]:
########################################################################################################################
# Export the result as an XLSX file
########################################################################################################################
os.makedirs(DICT_OUT_DIR_PATH, exist_ok=True)
out_xlsx_path: str = os.path.join(DICT_OUT_DIR_PATH, 'Dictionary_v1.xlsx')
with pd.ExcelWrriter(out_xlsx_path) as writer:
    df_pat_dict.to_excel(writer, sheet_name='Patient', index=False)